In [1]:
!pip install --upgrade --force-reinstall mediapipe opencv-python scipy numpy


  Using cached mediapipe-0.10.21-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached scipy-1.16.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached jax-0.7.2-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.7.2-cp312-cp312-manylinux_2_27_x86_64.whl.metadata (1.3 kB)
  Using cached matplotlib-3.10.6-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached opencv_contrib_p

In [2]:
!pip install mediapipe opencv-python

import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist


/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.3 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


In [8]:
# ==============================
# Step 2: Import Libraries
# ==============================
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist
from google.colab import drive
import os

# ==============================
# Step 3: Define EAR (Eye Aspect Ratio)
# ==============================
def eye_aspect_ratio(eye_landmarks):
    A = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    B = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    C = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (A + B) / (2.0 * C)
    return ear

# ==============================
# Step 4: Initialize Mediapipe Face Mesh
# ==============================
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

# ==============================
# Step 5: Define Thresholds and Counters
# ==============================
EAR_THRESHOLD = 0.25   # Eye closed threshold
CONSEC_FRAMES = 20     # Frames for drowsiness detection
counter = 0

# ==============================
# Step 6: Mount Google Drive
# ==============================
drive.mount('/content/drive')

# ==============================
# Step 7: Define Paths (Update Yours)
# ==============================
input_video = "/content/drive/MyDrive/AI ML Projects/Driver-Monitoring-System-YOLOv8/driver drowsiness/istockphoto-1807812249-640_adpp_is.mp4"
output_video = "/content/drive/MyDrive/AI ML Projects/Driver-Monitoring-System-YOLOv8/driver_drowsiness_output.mp4"

# Make sure folder exists
os.makedirs(os.path.dirname(output_video), exist_ok=True)

# ==============================
# Step 8: Process Video
# ==============================
cap = cv2.VideoCapture(input_video)
if not cap.isOpened():
    raise FileNotFoundError("❌ Could not open input video. Check the path!")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)) or 24  # default fps if 0

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape

            # Eye landmarks (indices for Mediapipe)
            right_eye = [(int(face_landmarks.landmark[i].x * w), int(face_landmarks.landmark[i].y * h))
                         for i in [33, 160, 158, 133, 153, 144]]
            left_eye = [(int(face_landmarks.landmark[i].x * w), int(face_landmarks.landmark[i].y * h))
                        for i in [263, 387, 385, 362, 380, 373]]

            left_EAR = eye_aspect_ratio(left_eye)
            right_EAR = eye_aspect_ratio(right_eye)
            avg_EAR = (left_EAR + right_EAR) / 2.0

            # Draw eye landmarks
            for (x, y) in left_eye + right_eye:
                cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

            # Drowsiness alert logic
            if avg_EAR < EAR_THRESHOLD:
                counter += 1
            else:
                counter = 0

            if counter >= CONSEC_FRAMES:
                cv2.putText(frame, "DROWSINESS ALERT!", (100, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print("✅ Detection complete! Output saved to:", output_video)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Detection complete! Output saved to: /content/drive/MyDrive/AI ML Projects/Driver-Monitoring-System-YOLOv8/driver_drowsiness_output.mp4
